In [2]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19880 sha256=bc96d23bd0dfa039024298f7a15443282364a84e050542b85a7009ee5d579845
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [3]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.linear_model import LogisticRegression
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Mounted at /gdrive
/gdrive


In [7]:


trainfile = r'/gdrive/My Drive/Assignment 3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/Assignment 3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.head()) 



(65000, 596)
   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]


In [8]:
print(test_data.shape)
print(test_data.head())

(173836, 596)
   CoverageField11A  CoverageField11B  ...  GeographicField64_TX  GeographicField64
0                13                22  ...                     0                 IL
1                 4                 5  ...                     0                 NJ
2                 3                 3  ...                     0                 NJ
3                 5                 9  ...                     0                 TX
4                12                21  ...                     0                 CA

[5 rows x 596 columns]


In [9]:
train_cat_cols = train_data.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(train_cat_cols.shape)
print(train_cat_cols)
test_cat_cols = test_data.select_dtypes(exclude=['float','int']).columns #selecting the categorical columns
print(test_cat_cols.shape)
print(test_cat_cols)

(0,)
Index([], dtype='object')
(1,)
Index(['GeographicField64'], dtype='object')


In [10]:
test_data=test_data.drop(columns=["GeographicField64"])

In [14]:
train_data.isnull().sum()


CoverageField11A        0
CoverageField11B        0
CoverageField1A         0
CoverageField1B         0
CoverageField2A         0
                       ..
GeographicField64_CA    0
GeographicField64_IL    0
GeographicField64_NJ    0
GeographicField64_TX    0
QuoteConversion_Flag    0
Length: 596, dtype: int64

In [15]:
test_data.isnull().sum()

CoverageField11A        0
CoverageField11B        0
CoverageField1A         0
CoverageField1B         0
CoverageField2A         0
                       ..
GeographicField63_Y     0
GeographicField64_CA    0
GeographicField64_IL    0
GeographicField64_NJ    0
GeographicField64_TX    0
Length: 595, dtype: int64

In [17]:

#Set dependent and independent variables
#Select dependent variables Column
X_train = train_data.iloc[:, :-1]
X_test = test_data.iloc[:, :]


#Select just Target Column
y_train = train_data.iloc[:, -1]


print(X_train.shape)
print(X_test.shape) 

print(y_train.shape)


(65000, 595)
(173836, 595)
(65000,)


In [18]:
print(train_data.head()) 

   CoverageField11A  ...  QuoteConversion_Flag
0                 2  ...                     0
1                 5  ...                     0
2                 4  ...                     0
3                15  ...                     0
4                 4  ...                     0

[5 rows x 596 columns]


In [19]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)



In [13]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_DT=pd.DataFrame(clf.predict(X_test),columns=["Prediction"] )

pred_DT.head()

,Prediction
0,0
1,1
2,0
3,0
4,1


In [ ]:
#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_DT_Probability =pd.DataFrame(clf.predict_proba(X_test))

pred_DT_Probability.head()

,0,1
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,0.0,1.0


In [ ]:
#Write into a file with actual prediction and corresponding probability
pd.concat([pred_DT,pred_DT],axis=1).to_csv("/gdrive/My Drive/Assignment 3/DT_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/DT_results.csv')
res.head()

,Prediction,Prediction.1
0,0,0
1,0,0
2,0,0
3,0,0
4,1,1


In [16]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)


In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_RF=pd.DataFrame(rfc.predict(X_test),columns=["Prediction"] )

pred_RF.head()

,Prediction
0,0
1,0
2,0
3,0
4,0


In [ ]:

pred_RF.head()
#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_RF_Probability =pd.DataFrame(rfc.predict_proba(X_test))

pred_RF_Probability.head()

,0,1
0,0.99,0.01
1,0.91,0.09
2,0.92,0.08
3,0.97,0.03
4,0.76,0.24


In [ ]:

pred_RF_Probability.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([pred_RF,pred_RF_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/RF_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/RF_results.csv')
res.head()

,Prediction,0,1
0,0,0.99,0.01
1,0,0.91,0.09
2,0,0.92,0.08
3,0,0.97,0.03
4,0,0.76,0.24


In [34]:
#Gradient Boosting ============================================================
abc =GradientBoostingClassifier()
abc.fit(X_train, y_train)
abc_predict=abc.predict(X_test)


In [35]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_abc=pd.DataFrame(abc.predict(X_test),columns=["Prediction"] )

#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_abc_Probability =pd.DataFrame(abc.predict_proba(X_test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_abc,pred_abc_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/GB_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/GB_results.csv')
res.head()


,Prediction,0,1
0,0,0.988995,0.011005
1,0,0.938972,0.061028
2,0,0.940773,0.059227
3,0,0.961426,0.038574
4,0,0.817220,0.182780


In [29]:
#Construct K-nearest Neighbour classifier 
# Default mode
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn_predict=knn.predict(X_test)

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_knn=pd.DataFrame(knn.predict(X_test),columns=["Prediction"] )


#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_knn_Probability =pd.DataFrame(knn.predict_proba(X_test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_knn,pred_knn_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/KNN_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/KNN_results.csv')
res.head()

,Prediction,0,1
0,0,0.8,0.2
1,0,0.8,0.2
2,0,0.6,0.4
3,0,1.0,0.0
4,0,0.6,0.4


In [33]:
#Construct Logistic regression classifier 
lgrg = LogisticRegression()
lgrg.fit(X_train, y_train)
lgrg_predict=lgrg.predict(X_test)

In [37]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_lgrg=pd.DataFrame(lgrg.predict(X_test),columns=["Prediction"] )


#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_lgrg_Probability =pd.DataFrame(lgrg.predict_proba(X_test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_lgrg,pred_lgrg_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/LogReg_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/LogReg_results.csv')
res.head()

,Prediction,0,1
0,0,0.928384,0.071616
1,0,0.776517,0.223483
2,0,0.763161,0.236839
3,0,0.975103,0.024897
4,0,0.797787,0.202213


In [ ]:
#Construct Multilayer Perceptron using the best parameters
#Default mode
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
mlp_predict=mlp.predict(X_test)

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_mlp=pd.DataFrame(mlp.predict(X_test),columns=["Prediction"] )

pred_mlp.head()

#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_mlp_Probability =pd.DataFrame(mlp.predict_proba(X_test))

pred_mlp_Probability.head()
#Write into a file with actual prediction and corresponding probability
pd.concat([pred_mlp,pred_mlp_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/MLP_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/MLP_results.csv')
res.head()

In [ ]:
#Construct Support Vector Machines using the best parameters
#Default mode
svc = svm.SVC()
svc.fit(X_train, y_train)
svc_predict=svc.predict(X_test)


In [ ]:
model_svc = LinearSVC()
m_svm = CalibratedClassifierCV(model_svc) 
m_svm.fit(X_train, y_train)
m_svm_predict=m_svm.predict(X_test)

In [ ]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_svc=pd.DataFrame(m_svm.predict(X_test),columns=["Prediction"] )


#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_svc_Probability =pd.DataFrame(m_svm.predict_proba(X_test))


#Write into a file with actual prediction and corresponding probability
pd.concat([pred_svc,pred_svc_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/SVM_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/SVM_results.csv')
res.head()

In [20]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [54]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, KNeighborsClassifier, LogisticRegression RandomForest and Decision Tree Classifier\n")

models = [ RandomForestClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier(), KNeighborsClassifier(), LogisticRegression()]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, KNeighborsClassifier, LogisticRegression RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [RandomForestClassifier]
    fold  0:  [0.93606674]
    fold  1:  [0.93504060]
    fold  2:  [0.93222683]
    fold  3:  [0.93111141]
    ----
    MEAN:     [0.93361140] + [0.00201484]
    FULL:     [0.93361144]

model  1:     [DecisionTreeClassifier]
    fold  0:  [0.91264388]
    fold  1:  [0.91491925]
    fold  2:  [0.91040914]
    fold  3:  [0.90978450]
    ----
    MEAN:     [0.91193919] + [0.00202244]
    FULL:     [0.91193923]

model  2:     [GradientBoostingClassifier]
    fold  0:  [0.94391898]
    fold  1:  [0.94423128]
    fold  2:  [0.94061482]
    fold  3:  [0.93865168]
    ----
    MEAN:     [0.94185419] + [0

In [55]:
#STACKING - CONTRUCT A GRADIENT BOOSTING MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)


In [56]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

,0,1
0,0.978169,0.021831
1,0.939377,0.060623
2,0.933813,0.066187
3,0.978169,0.021831
4,0.978169,0.021831


In [58]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_model=pd.DataFrame(model.predict(S_Test),columns=["Prediction"] )

#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_model_Probability =pd.DataFrame(model.predict_proba(S_Test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_model,pred_model_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/Final_Model2_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/Final_Model2_results.csv')
res.head()

,Prediction,0,1
0,0,0.978169,0.021831
1,0,0.939377,0.060623
2,0,0.933813,0.066187
3,0,0.978169,0.021831
4,0,0.978169,0.021831


In [48]:
#Gradient Boosting ============================================================
search_grid={'n_estimators':[5,10,20, 30, 50],'learning_rate':[0.01,.1]}
model =GradientBoostingClassifier()

model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)


#Randomized Search for hyperparameter tuning
model_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
model_random.fit(X_train, y_train)
grid_parm_model=model_random.best_params_
print(grid_parm_model)

{'n_estimators': 50, 'learning_rate': 0.1}


In [51]:
#STACKING - CONSTRUCT A GRADIENT BOOSTING MODEL using best parameter=============================
model_hpt =GradientBoostingClassifier(n_estimators= 50, learning_rate=0.1,)
    
model_hpt = model_hpt.fit(S_Train, y_res)
y_pred = model_hpt.predict(S_Test)

In [52]:
#Testset prediction======================================================================
#Get Class Prediction as a data frame with header as Prediction
pred_model_hpt=pd.DataFrame(model_hpt.predict(S_Test),columns=["Prediction"] )

#Get Class Prediction probabilities as a data frame 
#Get Prediction Probability for the predicted class as a dataframe
pred_model_hpt_Probability =pd.DataFrame(model_hpt.predict_proba(S_Test))

#Write into a file with actual prediction and corresponding probability
pd.concat([pred_model_hpt,pred_model_hpt_Probability],axis=1).to_csv("/gdrive/My Drive/Assignment 3/Final_Model_HPT_results.csv", index = None)

res=pd.read_csv('/gdrive/My Drive/Assignment 3/Final_Model_HPT_results.csv')
res.head()

,Prediction,0,1
0,0,0.965345,0.034655
1,0,0.910327,0.089673
2,0,0.924647,0.075353
3,0,0.965345,0.034655
4,0,0.965345,0.034655
